# COGS 108 - Data Checkpoint

# Names

- Ryan Swartz
- Egor Pustovalov
- Matthew Cohen
- Xuwen Yan(Ella)
- Connor McManigal

<a id='research_question'></a>
# Research Question

Is there an association between an NBA team’s number and rank of injured players to their record from the 2010 season through the 2015 season? Also, can we accurately predict a team’s change in record based on injuries that occur in future seasons?

# Dataset(s)

# First Dataset - Injury Stats
### Link to Dataset: https://www.kaggle.com/datasets/ghopkins/nba-injuries-2010-2018
#### No. Observations: 17,408 
This dataset describes the occurance, injury type, player team, and time of injury for NBA players in the 2010 - 2020 years. 

The information present here will be combined with the Historical NBA Performance dataset to match players with their teams for a given year. This will allow us to show how the number of injuries across different players for the same team impacts the team's historical performance.

Player Injury Table
|Columns | Data Types| | -------|-----------| |Date |date | |Team |varchar | |Relinquished| varchar| |Notes | varchar|

In this table the relinquished column states the player’s name and the Notes column states the injury type. Unfortunately, the table is not atomic because the Notes lists multiple injuries that may occur. This will have to be processed during the analysis.

# Second Dataset - Historical NBA Performance
### Link to Dataset: https://data.world/gmoney/nba-team-records-by-year
#### No. Observations: 208
This dataset describes the number of wins, number of losses, and winning percentage of an NBA team in a given year. 

The information present here will be combined with the injury dataset to match players with their teams present in this dataset for a given year. This will allow us to show how the number of injuries across different players for the same team impacts the team's historical performance. 

In order to answer how the rank of injured players impacts a team’s ranking, there are several variables and relationships that need to exist in the data. Our dataset must contain tables that include player to injury, player to player ranking/stats, player to their team, and team to ranking/stats associations.
While having multiple thousands of data points would be wonderful, our experiment is unfortunately constrained by the injuries sustained during playtime. Each year of basketball in the NBA contains around 1,230 matches, and according to a study by the National Library of Medicine, there are 19.1 athlete injuries per 1,000 exposures. Therefore, there should be on average 1,230 matches 30 athletes 0.0191 injuries per match = 704 athlete injuries per year ( two 15 player teams are in each match, giving us 30 athletes). Since we are examining 5 years of NBA data, we should get roughly 3,520 injuries in that timeframe. Most of this data will be collected from NBA Stuffer which provides free data on players, teams, etc. Player injury information will come from an NBA Injuries from 2010-2020 dataset available on kaggle.
Currently there are three main tables from these sources that will be used to investigate the hypothesis. More may be added later on. Ideally we would normalize the tables into 3NF to ensure lossless join and dependency preservation based on functional dependencies that we will establish. Our data may then be stored in an SQL database such as SQLite and read into python through the sqlite3 library interface. Below is a summary of each table.

# Setup

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

from IPython.display import display_html 

In [2]:
import pip
pip.main(["install", "openpyxl"])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable


0

## Data Cleaning - First Dataset

In [3]:
#first dataset - Injury stats
injury = pd.read_csv('injuries_2010-2020.csv')
injury

,Date,Team,Acquired,Relinquished,Notes
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely)
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely)
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely)
...,...,...,...,...,...
27100,2020-09-30,Lakers,Dion Waiters,NaN,activated from IL
27101,2020-10-02,Heat,NaN,Bam Adebayo,strained neck (DTD)
27102,2020-10-02,Heat,NaN,Goran Dragic,placed on IL with torn plantar fascia in left ...
27103,2020-10-02,Heat,Chris Silva,NaN,activated from IL


The NBA season roughly goes from October until June. So we are interpretting the column 'Season' as the year the season started ie. any injury happening in the 2015-2016 season will be read as the Season: 2015.

### sub-datasets

#### convert Date(string) to numeric value year(int) to make convinient for late analysis

In [4]:
injury = injury.assign(
    Year = injury.get('Date').apply(lambda s: int(s.split('-')[0]))
)
injury = injury[2010 <= injury.get('Year')]\
                       [(injury[2010 <= injury.get('Year')]).get('Year')<=2015]
injury = injury.assign(
    Season = injury.get('Date').apply(lambda s: int(s.split('-')[0])- np.where(int(s.split('-')[1]) < 9, 1, 0)))
injury

,Date,Team,Acquired,Relinquished,Notes,Year,Season
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...,2010,2010
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely),2010,2010
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...,2010,2010
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely),2010,2010
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely),2010,2010
...,...,...,...,...,...,...,...
14072,2015-12-31,Timberwolves,NaN,Kevin Garnett,placed on IL for rest,2015,2015
14073,2015-12-31,Timberwolves,Damjan Rudez,NaN,activated from IL,2015,2015
14074,2015-12-31,Warriors,NaN,Leandro Barbosa,sprained left shoulder (DTD),2015,2015
14075,2015-12-31,Pacers,Rakeem Christmas,NaN,activated from IL,2015,2015


since we notice that there are total 33 unique teams in the dataset, it is not convinient to create a subset for each team. Hence, we choose to create a subset and group the dataset by year and team.

<code>injury.get('Team').unique()<code>
- array(['Bulls', 'Pistons', 'Blazers', 'Nets', 'Nuggets', 'Bucks', 'Kings',
       'Bobcats', 'Warriors', 'Suns', 'Heat', 'Thunder', 'Timberwolves',
       'Celtics', 'Lakers', 'Rockets', '76ers', 'Cavaliers', 'Clippers',
       'Grizzlies', 'Hawks', 'Hornets', 'Jazz', 'Knicks', 'Mavericks',
       'Pacers', 'Raptors', 'Spurs', 'Magic', 'Wizards', 'Pelicans', nan,
       'Bullets'], dtype=object)

Craete a subset to see how many Acquired and Relinquished injuries for each team in each year.

Since our hypothesis focus on the seasons in 2015-2019, we filter out the dataset to make it only contains the data from 2015-2019.

In [5]:
grouped_by_team_year = injury.groupby(['Year','Team']).count().drop(columns=['Date'])
grouped_by_team_year

Acquired  Relinquished  Notes  Season
Year Team                                               
2010 76ers               12            14     26      26
     Blazers              4            25     29      29
     Bobcats             16            21     37      37
     Bucks                7            40     47      47
     Bulls               11            17     28      28
...                     ...           ...    ...     ...
2015 Suns                46            58    104     104
     Thunder             27            39     66      66
     Timberwolves        38            60     98      98
     Warriors            49            58    107     107
     Wizards             36            51     87      87

[182 rows x 4 columns]

### Clean out NaN value in Acquired and Relinquished columns separately

First, create subset that clean out the NaN value in Acquired column.

In [6]:
acquired_injury = injury.dropna(subset=['Acquired'])
acquired_injury

,Date,Team,Acquired,Relinquished,Notes,Year,Season
53,2010-10-27,Heat,Jerry Stackhouse,NaN,activated from IL,2010,2010
81,2010-10-27,Rockets,Jermaine Taylor,NaN,activated from IL,2010,2010
101,2010-10-29,Cavaliers,Samardo Samuels,NaN,activated from IL,2010,2010
103,2010-10-29,Celtics,Luke Harangody,NaN,activated from IL,2010,2010
105,2010-10-29,Grizzlies,Marc Gasol,NaN,activated from IL,2010,2010
...,...,...,...,...,...,...,...
14068,2015-12-31,Rockets,Terrence Jones,NaN,returned to lineup,2015,2015
14071,2015-12-31,Thunder,Mitch McGary,NaN,activated from IL,2015,2015
14073,2015-12-31,Timberwolves,Damjan Rudez,NaN,activated from IL,2015,2015
14075,2015-12-31,Pacers,Rakeem Christmas,NaN,activated from IL,2015,2015


Next, we group the subset by year and team to see how many acquired datapoints per team in each year.

In [7]:
acquired_injury.groupby(['Year','Team']).count().drop(columns=['Date','Notes','Relinquished'])

Acquired  Season
Year Team                          
2010 76ers               12      12
     Blazers              4       4
     Bobcats             16      16
     Bucks                7       7
     Bulls               11      11
...                     ...     ...
2015 Suns                46      46
     Thunder             27      27
     Timberwolves        38      38
     Warriors            49      49
     Wizards             36      36

[182 rows x 2 columns]

We also create a subset that clean out the NaN value in Relinquished column.

In [8]:
relinquished_injury = injury.dropna(subset=['Relinquished'])
relinquished_injury

,Date,Team,Acquired,Relinquished,Notes,Year,Season
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...,2010,2010
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely),2010,2010
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...,2010,2010
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely),2010,2010
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely),2010,2010
...,...,...,...,...,...,...,...
14066,2015-12-31,Pacers,NaN,Ian Mahinmi,placed on IL with knee injury,2015,2015
14069,2015-12-31,Suns,NaN,Jon Leuer,ankle injury (DTD),2015,2015
14070,2015-12-31,Thunder,NaN,Steve Novak,placed on IL,2015,2015
14072,2015-12-31,Timberwolves,NaN,Kevin Garnett,placed on IL for rest,2015,2015


Then, we group the subset by year and team to see how many relinquished datapoints per team in each year.

In [9]:
relinquished_injury.groupby(['Year','Team']).count().drop(columns=['Date','Notes','Acquired'])

Relinquished  Season
Year Team                              
2010 76ers                   14      14
     Blazers                 25      25
     Bobcats                 21      21
     Bucks                   40      40
     Bulls                   17      17
...                         ...     ...
2015 Suns                    58      58
     Thunder                 39      39
     Timberwolves            60      60
     Warriors                58      58
     Wizards                 51      51

[182 rows x 2 columns]

#### create subsets of the data based on certain positions of the injuries

In [10]:
knee = injury[injury['Notes'].str.contains("knee")]\
                .groupby(['Year','Team']).count().drop(columns=['Date','Notes'])
ankle = injury[injury['Notes'].str.contains("ankle")]\
                .groupby(['Year','Team']).count().drop(columns=['Date','Notes'])
foot = injury[injury['Notes'].str.contains("foot")]\
                .groupby(['Year','Team']).count().drop(columns=['Date','Notes'])

hand = injury[injury['Notes'].str.contains("hand")]\
                .groupby(['Year','Team']).count().drop(columns=['Date','Notes'])
finger = injury[injury['Notes'].str.contains("finger")]\
                .groupby(['Year','Team']).count().drop(columns=['Date','Notes'])
back = injury[injury['Notes'].str.contains("back")]\
                .groupby(['Year','Team']).count().drop(columns=['Date','Notes'])


knee_styler = knee.head().style.set_table_attributes("style='display:inline'").set_caption('injuries related to knee')
ankle_styler = ankle.head().style.set_table_attributes("style='display:inline'").set_caption('injuries related to ankle')
foot_styler = foot.head().style.set_table_attributes("style='display:inline'").set_caption('injuries related to foot')
finger_styler = finger.head().style.set_table_attributes("style='display:inline'").set_caption('injuries related to finger')
back_styler = back.head().style.set_table_attributes("style='display:inline'").set_caption('injuries related to back')
hand_styler = hand.head().style.set_table_attributes("style='display:inline'").set_caption('injuries related to hand')

display_html(knee_styler._repr_html_()+\
             ankle_styler._repr_html_()+\
             foot_styler._repr_html_()+\
             finger_styler._repr_html_()+\
             back_styler._repr_html_()+\
             hand_styler._repr_html_(), raw=True)


injuries related to knee 
 
 
   
   
 Acquired 
 Relinquished 
 Season 
 
 
 Year 
 Team 
   
   
   
 
 
 
 
 2010 
 Blazers 
 0 
 15 
 15 
 
 
 Bobcats 
 0 
 1 
 1 
 
 
 Bucks 
 0 
 1 
 1 
 
 
 Cavaliers 
 0 
 1 
 1 
 
 
 Celtics 
 0 
 6 
 6 
 
 
 

 
 injuries related to ankle 
 
 
   
   
 Acquired 
 Relinquished 
 Season 
 
 
 Year 
 Team 
   
   
   
 
 
 
 
 2010 
 Blazers 
 0 
 3 
 3 
 
 
 Bobcats 
 0 
 2 
 2 
 
 
 Bucks 
 0 
 3 
 3 
 
 
 Bulls 
 0 
 1 
 1 
 
 
 Celtics 
 0 
 6 
 6 
 
 
 

 
 injuries related to foot 
 
 
   
   
 Acquired 
 Relinquished 
 Season 
 
 
 Year 
 Team 
   
   
   
 
 
 
 
 2010 
 Bucks 
 0 
 9 
 9 
 
 
 Bulls 
 0 
 1 
 1 
 
 
 Heat 
 0 
 2 
 2 
 
 
 Kings 
 0 
 1 
 1 
 
 
 Magic 
 0 
 6 
 6 
 
 
 

 
 injuries related to finger 
 
 
   
   
 Acquired 
 Relinquished 
 Season 
 
 
 Year 
 Team 
   
   
   
 
 
 
 
 2010 
 Bulls 
 0 
 2 
 2 
 
 
 Cavaliers 
 0 
 1 
 1 
 
 
 Kings 
 0 
 1 
 1 
 
 
 Suns 
 0 
 1 
 1 
 
 
 Warriors 
 0 
 2 
 2 
 
 
 

 
 injuries related to back 
 
 
   
   
 Acquired 
 Relinquished 
 Season 
 
 
 Year 
 Team 
   
   
   
 
 
 
 
 2010 
 Blazers 
 0 
 1 
 1 
 
 
 Bucks 
 0 
 6 
 6 
 
 
 Clippers 
 0 
 3 
 3 
 
 
 Grizzlies 
 0 
 1 
 1 
 
 
 Hawks 
 0 
 3 
 3 
 
 
 

 
 injuries related to hand 
 
 
   
   
 Acquired 
 Relinquished 
 Season 
 
 
 Year 
 Team 
   
   
   
 
 
 
 
 2010 
 Heat 
 0 
 2 
 2 
 
 
 2011 
 Blazers 
 0 
 1 
 1 
 
 
 2012 
 Bucks 
 0 
 3 
 3 
 
 
 Cavaliers 
 0 
 2 
 2 
 
 
 Clippers 
 0 
 2 
 2

# Data Cleaning - Second Dataset

In [11]:
#second dataset - Team Performance
performance = pd.read_excel('Historical_NBA_Performance.xlsx')
performance

,Year,Team,Record,Winning Percentage,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,2016-17,Celtics,25-15,0.625,NaN,NaN,NaN,NaN,NaN,NaT
1,2015-16,Celtics,48-34,0.585,NaN,NaN,NaN,NaN,NaN,NaT
2,2014-15,Celtics,40-42,0.488,NaN,NaN,NaN,NaN,NaN,NaT
3,2013-14,Celtics,25-57,0.305,NaN,NaN,NaN,NaN,NaN,NaT
4,2012-13,Celtics,41-40,0.506,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...
1412,1965-66,Bullets,38-42,0.475,NaN,NaN,NaN,NaN,NaN,NaT
1413,1964-65,Bullets,37-43,0.463,NaN,NaN,NaN,NaN,NaN,NaT
1414,1963-64,Bullets,31-49,0.388,NaN,NaN,NaN,NaN,NaN,NaT
1415,1962-63,Zephyrs,25-55,0.313,NaN,NaN,NaN,NaN,NaN,NaT


## Data Cleaning - Second Dataset

Let performance_clean be the clean dataset. 

In [12]:
performance_clean = performance[['Year', 'Team', 'Record', 'Winning Percentage']]
print(performance_clean.get('Team').unique())
performance_clean.head()

['Celtics' 'Hawks' 'Blackhawks' 'Nets' 'Hornets' 'Bobcats' 'Bulls'
 'Cavaliers' 'Mavericks' 'Nuggets' 'Pistons' 'Warriors' 'Rockets' 'Pacers'
 'Clippers' 'Braves' 'Lakers' 'Grizzlies' 'Heat' 'Bucks' 'Timberwolves'
 'Pelicans' 'Knicks' 'Thunder' 'Supersonics' 'Magic' '76ers' 'Nationals'
 'Suns' 'Trail Blazers' 'Kings' 'Royals' 'Spurs' 'Raptors' 'Jazz'
 'Wizards' 'Bullets' 'Zephyrs' 'Packers']


,Year,Team,Record,Winning Percentage
0,2016-17,Celtics,25-15,0.625
1,2015-16,Celtics,48-34,0.585
2,2014-15,Celtics,40-42,0.488
3,2013-14,Celtics,25-57,0.305
4,2012-13,Celtics,41-40,0.506


In [13]:
#changes Year column to Season column
performance_clean.rename(columns = {'Year': 'Season'}, inplace = True)
performance_clean.head()

/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Season,Team,Record,Winning Percentage
0,2016-17,Celtics,25-15,0.625
1,2015-16,Celtics,48-34,0.585
2,2014-15,Celtics,40-42,0.488
3,2013-14,Celtics,25-57,0.305
4,2012-13,Celtics,41-40,0.506


In [14]:
#adds columnn Year using indexing on Season column
performance_clean['Year'] = performance_clean['Season'].astype(str).apply(lambda s: s.split('-')[0])


#filters Year column to grab Seasons 2010-2015
performance_clean['Year'] = performance_clean['Year'].astype(int)

performance_clean = performance_clean[2010 <= performance_clean.get('Year')]\
                     [(performance_clean[2010 <= performance_clean.get('Year')]).get('Year')<=2015]

performance_clean.head()


/tmp/ipykernel_269/778074175.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performance_clean['Year'] = performance_clean['Season'].astype(str).apply(lambda s: s.split('-')[0])
/tmp/ipykernel_269/778074175.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performance_clean['Year'] = performance_clean['Year'].astype(int)


,Season,Team,Record,Winning Percentage,Year
1,2015-16,Celtics,48-34,0.585,2015
2,2014-15,Celtics,40-42,0.488,2014
3,2013-14,Celtics,25-57,0.305,2013
4,2012-13,Celtics,41-40,0.506,2012
5,2011-12 *,Celtics,39-27,0.591,2011


In [15]:
#cleans irregular values of Season column for the years 2010-11 and 2011-12
def fix_season(s):
    if (s == '2011-12 *'):
        return '2011-12'
    elif (s == datetime.datetime(2010, 11, 1, 0, 0)):
        return '2010-11'
    else:
        return s

performance_clean['Season'] = performance_clean['Season'].apply(fix_season)

In [16]:
def win(s):
    return s.split('-')[0]
def loss(s):
    return s.split('-')[1]
performance_clean['Win'] = performance_clean['Record'].apply(win)
performance_clean['Loss'] = performance_clean['Record'].apply(loss)
performance_clean.drop(['Record'], axis=1)

,Season,Team,Winning Percentage,Year,Win,Loss
1,2015-16,Celtics,0.585,2015,48,34
2,2014-15,Celtics,0.488,2014,40,42
3,2013-14,Celtics,0.305,2013,25,57
4,2012-13,Celtics,0.506,2012,41,40
5,2011-12,Celtics,0.591,2011,39,27
...,...,...,...,...,...,...
1363,2014-15,Wizards,0.561,2014,46,36
1364,2013-14,Wizards,0.537,2013,44,38
1365,2012-13,Wizards,0.354,2012,29,53
1366,2011-12,Wizards,0.303,2011,20,46


To combine our two datasets: we will be linking the season columns together. For the injury dataset, this will be the start year of the season (ie 2013), and for the performance dataset, this will be the range (2013-14).

In [17]:
#merge the 2 datasets by season
performance_clean.merge(injury, left_on='Year', right_on='Season').head()

,Season_x,Team_x,Record,Winning Percentage,Year_x,Win,Loss,Date,Team_y,Acquired,Relinquished,Notes,Year_y,Season_y
0,2015-16,Celtics,48-34,0.585,2015,48,34,2015-09-02,NaN,NaN,Glen Davis,surgery on left ankle to repair torn ligaments...,2015,2015
1,2015-16,Celtics,48-34,0.585,2015,48,34,2015-09-02,Grizzlies,NaN,Jarell Martin,fractured left foot (surgery) (out indefinitely),2015,2015
2,2015-16,Celtics,48-34,0.585,2015,48,34,2015-09-03,NaN,NaN,Landry Fields,surgery on hip to repair torn labrum (out inde...,2015,2015
3,2015-16,Celtics,48-34,0.585,2015,48,34,2015-09-03,Jazz,NaN,Dante Exum,surgery on left knee to repair torn ACL (out i...,2015,2015
4,2015-16,Celtics,48-34,0.585,2015,48,34,2015-09-22,Pistons,NaN,Darrun Hilliard,surgery to repair fractured nose (out indefini...,2015,2015
